<a href="https://colab.research.google.com/github/rodrigofer89/cybersecurity/blob/main/CyberSecurity_checkpoint1_semestre2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyngrok
!pip install -q streamlit
!pip install -q streamlit_ace
!pip install captcha
!pip install git+https://github.com/cocodataset/panopticapi.git
!git clone https://github.com/facebookresearch/detectron2.git
!pip install ./detectron2
!pip install --upgrade pyyaml

     |████████████████████████████████| 745 kB 25.0 MB/s 
     |████████████████████████████████| 9.1 MB 28.3 MB/s 
     |████████████████████████████████| 164 kB 68.4 MB/s 
     |████████████████████████████████| 78 kB 8.5 MB/s 
     |████████████████████████████████| 181 kB 67.6 MB/s 
     |████████████████████████████████| 235 kB 65.5 MB/s 
     |████████████████████████████████| 4.7 MB 66.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 51 kB 6.4 MB/s 
     |████████████████████████████████| 3.6 MB 34.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 13.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-93gbhynj
  Running command git clone -q https://github.com/cocodataset/panopticapi.git /tmp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!streamlit run app.py &>/dev/null&

In [3]:
%%writefile app.py

import re
import json
import requests
import streamlit as st
import cv2
from pyngrok import ngrok
from captcha.image import ImageCaptcha
import matplotlib.pyplot as plt
import numpy as np
import random
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog

import string
characters = string.digits + string.ascii_uppercase

generator = ImageCaptcha(320, 100)

if 'img' not in st.session_state:
  st.session_state["random_str"] = ''.join([random.choice(characters) for i in range(6)])
  st.session_state["img"] = generator.generate_image(st.session_state["random_str"])

#Essa função é para criação da webpage
def main():
  #Elementos da webpage
  html_temp = """
  <div>
    <div style ="width: 34%; margin-right: 1%; height: 100px; display: inline-block; float: left; background-image: url(https://www.santander.com.br/sites/WPC_CMS/imagem/22-08-16_175326_M_logo-santander.svg); background-repeat: no-repeat; background-position-x: center; background-position-y: center; background-size: 90%;">
    </div>
    <div style ="background: #CC0100; width: 65%; height: 100px; display: inline-block; padding: 23px;">
      <h3 style ="color: white; text-align: center;">Novo Cadastro<h3>
    </div>
  </div>
  """

  # Função do stramlit que faz o display da webpage
  st.markdown(html_temp, unsafe_allow_html = True) 

  # As linhas abaixo criam as caixas na qual o usuário vai entrar com dados da pessoa que quer o empréstimo para fazer a Predição
  col1, col2 = st.columns(2)
  with col1:
    nome = st.text_input('Nome*')

  with col2:
    sobrenome = st.text_input('Sobrenome*')

  col3, col4 = st.columns(2)
  with col3:
    email = st.text_input('Email*')

  with col4:
    telefone = st.text_input('Telefone*')

  cpf = st.text_input('CPF*')

  uploaded_file = st.file_uploader("Selecione uma selfie com um relógio*", help="A sua selfie com um objeto é um método de segurança para futuras operações no aplicativo")

  col5, col6 = st.columns(2)
  with col5:
    imagem = st.image(st.session_state["img"])
  
  with col6:
    textCaptcha = st.text_input("Captcha")

  # Valida email
  def isEmailValid(email):
    regex = re.compile(r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+')
    if re.fullmatch(regex, email):
      return True
    else:
      return False

  def validation(nome, sobrenome, email, telefone, cpf):
    flg_cpf_cnpj_recusado = False

    if st.button("Cadastrar"): 
      if nome and sobrenome and isEmailValid(email) and telefone.isnumeric() and cpf.isnumeric():
        if uploaded_file:
          if validarCaptcha(textCaptcha):
            file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
            image = cv2.imdecode(file_bytes, 1)
            if validarImagem(image):
              st.success("Cadastro autorizado!")
            else:
              st.error("Cadastro não autorizado!")
          else:
            st.warning('Captcha inválido')  
        else:
          st.warning('Selecione uma selfie com o objeto solicitado')  
      else:
        st.warning('Digite os campos corretamente')
        return

  def validarCaptcha(textCaptcha):
    if textCaptcha.upper() == st.session_state["random_str"]:
      return True
    else:
      return False

  def validarImagem(image):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    predictor = DefaultPredictor(cfg)
    outputs = predictor(image)
    pred_classes = outputs['instances'].pred_classes.cpu().tolist()
    class_names = MetadataCatalog.get("coco_2017_val_panoptic_separated").thing_classes
    pred_class_names = list(map(lambda x: class_names[x], pred_classes))
    print(pred_class_names)
    try:
      if "person" in pred_class_names and "clock" in pred_class_names:
        return True
      return False
    except:
      return False

  validation(nome, sobrenome, email, telefone, cpf)

if __name__=='__main__': 
  main()

Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-09-08 16:47:58.810 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.86.213.191:8501

npx: installed 22 in 3.637s
your url is: https://famous-coats-kick-34-86-213-191.loca.lt
